In [2]:
from pyspark.sql import SparkSession

ICEBERG_VERSION="1.1.0"
ICEBERG_DEPENDENCY=f"org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:{ICEBERG_VERSION}"

AWS_SDK_VERSION="2.17.257"
AWS_MAVEN_GROUP="software.amazon.awssdk"
AWS_PACKAGES = ["bundle", "url-connection-client"]

DEPENDENCIES = [ICEBERG_DEPENDENCY] + [f"{AWS_MAVEN_GROUP}:{pkg}:{AWS_SDK_VERSION}" for pkg in AWS_PACKAGES]

spark = SparkSession.builder \
    .master("local") \
    .config("spark.jars.packages", ",".join(DEPENDENCIES)) \
    .config("spark.sql.catalog.glue_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.glue_catalog.warehouse", "s3://varokas-iceberg-demo/iceberg") \
    .config("spark.sql.catalog.glue_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config("spark.sql.catalog.glue_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.ui.enabled", "false") \
    .getOrCreate()

In [3]:
# https://www.kaggle.com/competitions/nyc-taxi-trip-duration/code
import pandas as pd
pdDF = pd.read_csv("test.zip")[["vendor_id", "pickup_datetime", "passenger_count"]]
pdDF['pickup_datetime'] = pd.to_datetime(pdDF['pickup_datetime'])

In [15]:
int(len(pdDF)/2)

312567

In [18]:
first_half = pdDF[:int(len(pdDF)/2)]
second_half = pdDF[int(len(pdDF)/2):]

In [19]:
df1 = spark.createDataFrame(first_half)

/Users/varokas/projects/iceberg-demo/.venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/varokas/projects/iceberg-demo/.venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [20]:
df1.writeTo("glue_catalog.iceberg_demo.nyc_taxis").append()

23/02/18 16:11:46 WARN TaskSetManager: Stage 1 contains a task of very large size (4591 KiB). The maximum recommended task size is 1000 KiB.


In [21]:
df2 = spark.createDataFrame(second_half)

In [22]:
df2.writeTo("glue_catalog.iceberg_demo.nyc_taxis").append()

23/02/18 16:13:29 WARN SparkWriteBuilder: Skipping distribution/ordering: extensions are disabled and spec contains unsupported transforms
23/02/18 16:13:29 WARN TaskSetManager: Stage 2 contains a task of very large size (4591 KiB). The maximum recommended task size is 1000 KiB.
